In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim


import torchtext
from torchtext.data import Field, TabularDataset
from torchtext.data import Iterator, BucketIterator

import gensim

In [12]:

# model = gensim.models.KeyedVectors.load_word2vec_format()
# weights = torch.FloatTensor(model.vectors)

In [13]:
glove = torchtext.vocab.GloVe(name="6B", dim=50)

In [14]:
x = glove['cat']
y = glove['dog']
torch.norm(y - x)

torch.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0))


tensor([0.9218])

In [15]:
word = 'cat'
other = ['dog', 'bike', 'kitten', 'puppy', 'kite', 'computer', 'neuron']
for w in other:
    dist = torch.norm(glove[word] - glove[w]) # euclidean distance
    print(w, float(dist))

dog 1.8846031427383423
bike 5.048375129699707
kitten 3.5068609714508057
puppy 3.0644655227661133
kite 4.210376262664795
computer 6.030652046203613
neuron 6.228669166564941


In [16]:
def print_closest_words(vec, n=5):
    dists = torch.norm(glove.vectors - vec, dim=1)     # compute distances to all words
    lst = sorted(enumerate(dists.numpy()), key=lambda x: x[1]) # sort by distance
    for idx, difference in lst[1:n+1]: # take the top n
        print(glove.itos[idx], difference)

print_closest_words(glove["cat"], n=10)

dog 1.8846031
rabbit 2.4572797
monkey 2.8102052
cats 2.8972247
rat 2.9455352
beast 2.9878407
monster 3.0022194
pet 3.0396757
snake 3.0617998
puppy 3.0644655


In [ ]:
# FloatTensor containing pretrained weights
# weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding = nn.Embedding.from_pretrained(weights)
# Get embeddings for index 1
input_ = torch.LongTensor([1])
embedding(input_)